# Fetching Weather Data and Uploading to AWS S3

This guide provides steps for fetching weather data using the OpenWeatherMap API and then uploading the data to an Amazon S3 bucket using Python.

## Step 1: Obtain an API Key from OpenWeatherMap

1. Register on the [OpenWeatherMap website](https://openweathermap.org/) and create an account.
2. Find and copy the API key from your account dashboard.

## Step 2: Write Python Function to Fetch Weather Data

In [1]:
import requests
import weather_key

def get_weather_data(city, api_key):
    base_url = "https://api.openweathermap.org/data/2.5/weather"
    params = {'q': city, 'appid': api_key}
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Failed to fetch weather data")

## Step 3: Set Up AWS Credentials for S3 Access
* Install Boto3 using pip install boto3.
* Configure AWS credentials (AWS Access Key ID and Secret Access Key).

In [2]:
#!pip install boto3

In [3]:
pip show Boto3

Name: boto3
Version: 1.34.40
Summary: The AWS SDK for Python
Home-page: https://github.com/boto/boto3
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: c:\users\ankit\anaconda3\envs\dl38\lib\site-packages
Requires: botocore, jmespath, s3transfer
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## Step 4: Write Python Function to Upload Data to S3


In [4]:
import boto3
import json

def upload_to_s3(bucket_name, file_name, data):
    s3 = boto3.client('s3',
            aws_access_key_id=weather_key.aws_access_key_id, 
            aws_secret_access_key= weather_key.aws_secret_access_key, region_name="us-east-1")
    s3.put_object(Bucket=bucket_name, Key=file_name, Body=json.dumps(data))

## Step 5: Combine the Functions in a Script


In [5]:
import importlib
import weather_key
importlib.reload(weather_key)

# Main execution
api_key = weather_key.api_key   # Replace with your API key
city = "London"  # Replace with desired city
bucket_name = "lab03ankit"  # Replace with your S3 bucket name
file_name = "weather_data.json"

try:
    weather_data = get_weather_data(city, api_key)
    upload_to_s3(bucket_name, file_name, weather_data)
    print("Data uploaded successfully to S3")
except Exception as e:
    print(f"An error occurred: {e}")

Data uploaded successfully to S3


In [6]:
# Example usage
api_key = weather_key.api_key  # Replace with your actual API key
city = "London"
try:
    weather_data = get_weather_data(city, api_key)
    print(weather_data)
except Exception as e:
    print(f"An error occurred: {e}")

{'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 288.03, 'feels_like': 287.51, 'temp_min': 287.06, 'temp_max': 288.71, 'pressure': 1006, 'humidity': 74}, 'visibility': 10000, 'wind': {'speed': 4.63, 'deg': 220}, 'clouds': {'all': 75}, 'dt': 1712340715, 'sys': {'type': 2, 'id': 2075535, 'country': 'GB', 'sunrise': 1712294752, 'sunset': 1712342396}, 'timezone': 3600, 'id': 2643743, 'name': 'London', 'cod': 200}


# Assignment: Groups to Convert JSON to CSV

Step 1. Take the JSON output and convert it to a Dataframe using pandas
Step 2. Now upload the CSV file to the 'lab-03' S3 bucket in the cloud with the following naming convention: <your group name>_weather_date_london_<datetimestamp>.csv


In [10]:
###### INSERT CODE BELOW ####

import json
import csv
import pandas as pd
from datetime import datetime
from io import StringIO
import weather_key

df = pd.json_normalize(weather_data)

def upload_file_to_s3(dataframe, city):
    s3 = boto3.client("s3", aws_access_key_id=weather_key.aws_access_key_id, aws_secret_access_key=weather_key.aws_secret_access_key)
    temp_df = StringIO()
    df.to_csv(temp_df, header=True, index=False)
    temp_df.seek(0)    
    current_datetime = datetime.now().strftime("%Y%m%d%H%M%S") 
    filename = f"weather_data_{city}_{current_datetime}.csv"  
    s3.put_object(Bucket="lab03ankit", Body=temp_df.getvalue(), Key=filename)
    print("File Uploaded Successfully to S3 Bucket.")

upload_file_to_s3(df, "london")
### END CODE ###

File Uploaded Successfully to S3 Bucket.
